In [1]:
import pandas as pd
import numpy as np
import collections
import re
import sqlite3

In [2]:
# Change this to your path to the DB file!
database = "flights.db"
conn = sqlite3.connect(database)

### Load raw data 

In [3]:
# load our datasets
# These first four are used as lookup tables
airlines =     pd.read_sql('SELECT * FROM airlines', con=conn)
airports =     pd.read_sql('SELECT * FROM airports', con=conn)
cancel_codes = pd.read_sql('SELECT * FROM cancel_codes', con=conn)
carriers =     pd.read_sql('SELECT * FROM carriers', con=conn)
# This has all the data about flights
flights =      pd.read_sql('SELECT * FROM flights', con=conn)

In [4]:
#Closing the connection to the db, since we're done using it
conn.close()

### Some transforms to get it ready for analysis

In [5]:
# Turn our flights table into a list of dicts
# Normally we wouldn't do this, but for the sake of learning to deal with data
# structures in Python, we'll do it this way :)
flights = flights.to_dict(orient='records')

In [6]:
type(flights)

list

In [7]:
#flights is currently a list of dicts technically stored in an array. 
#It has 410,517 elements in it.
print(len(flights))

410517


In [8]:
#viewing and inspecting the first element.
flights[0]

{'DAY_OF_MONTH': 1,
 'DAY_OF_WEEK': 3,
 'FL_DATE': '2017-02-01',
 'UNIQUE_CARRIER': 'B6',
 'AIRLINE_ID': 20409,
 'FL_NUM': 33,
 'ORIGIN_AIRPORT_ID': 10785,
 'ORIGIN_CITY_NAME': 'Burlington, VT',
 'ORIGIN_STATE_NM': 'Vermont',
 'DEST_AIRPORT_ID': 12478,
 'DEST_CITY_NAME': 'New York, NY',
 'DEST_STATE_NM': 'New York',
 'DEP_TIME': 1857.0,
 'DEP_DELAY': -10.0,
 'TAXI_OUT': 16.0,
 'WHEELS_OFF': 1913.0,
 'WHEELS_ON': 2019.0,
 'TAXI_IN': 10.0,
 'ARR_TIME': 2029.0,
 'ARR_DELAY': -8.0,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 0,
 'ACTUAL_ELAPSED_TIME': 92.0,
 'FLIGHTS': 1,
 'DISTANCE': 266,
 'CARRIER_DELAY': nan,
 'WEATHER_DELAY': nan,
 'NAS_DELAY': nan,
 'SECURITY_DELAY': nan,
 'LATE_AIRCRAFT_DELAY': nan,
 'Unnamed: 31': None}

In [9]:
{ row['CANCELLATION_CODE'] for row in flights }

{'A', 'B', 'C', None}

### HW 1: Answer the following questions about the `flights` :

What datatype is our `flights` object now (a list, dict, etc)?

How many elements are in it?

How can we look at/inspect the first element?

### HW 2: Create dictionaries from our lookup tables, making the `Code` the key and the `Description` the value

In [10]:
cancel_codes_dict= dict(zip(cancel_codes.Code, cancel_codes.Description))

# Go ahead and create more dictionaries for airlines, airports and carriers where
# for each the key will be the `Code` and the value will be the `Description`

airlines_dict = dict(zip(airlines.Code, airlines.Description))

airports_dict = dict(zip(airports.Code, airports.Description))

carriers_dict = dict(zip(carriers.Code, carriers.Description))
# When you're done there should be four dicts (cancel_codes_dict, airlines_dict, airports_dict, carriers_dict)

### HW 3: Add in our dictionary mappings to our `flights` data
Using our mappings, please create the following new keys in each 'row' of flights with the appropriate value:
 - AIRLINE_NAME
 - ORIGIN_AIRPORT_NAME
 - CARRIER_NAME
 - CANCELLATION_REASON

In [11]:
cancel_codes_dict

{'A': 'Carrier', 'B': 'Weather', 'C': 'National Air System', 'D': 'Security'}

In [12]:
for row_dict in flights:
    row_dict['CARRIER_NAME'] = carriers_dict.get(row_dict['UNIQUE_CARRIER'])
    row_dict['AIRLINE_NAME'] = airlines_dict.get(row_dict['AIRLINE_ID'])
    row_dict['ORIGIN_AIRPORT_NAME'] = airports_dict.get(row_dict['ORIGIN_AIRPORT_ID'])
    row_dict['CANCELLATION_REASON'] = cancel_codes_dict.get(row_dict['CANCELLATION_CODE'], 'No input')
    
   

In [13]:
#not right approch.
#for row_dict in flights:
    #if row_dict['CANCELLATION_CODE'] == 'None':
        #(row_dict['CANCELLATION_CODE'])
    #else:
        #cancellation_id = row_dict['CANCELLATION_CODE']
        #cancellation_nm = cancel_codes_dict[cancellation_id]
        #row_dict['CANCELLATION_REASON'] = cancel_codes_dict.get(row_dict['CANCELLATION_CODE'])

In [14]:
flights[0]

{'DAY_OF_MONTH': 1,
 'DAY_OF_WEEK': 3,
 'FL_DATE': '2017-02-01',
 'UNIQUE_CARRIER': 'B6',
 'AIRLINE_ID': 20409,
 'FL_NUM': 33,
 'ORIGIN_AIRPORT_ID': 10785,
 'ORIGIN_CITY_NAME': 'Burlington, VT',
 'ORIGIN_STATE_NM': 'Vermont',
 'DEST_AIRPORT_ID': 12478,
 'DEST_CITY_NAME': 'New York, NY',
 'DEST_STATE_NM': 'New York',
 'DEP_TIME': 1857.0,
 'DEP_DELAY': -10.0,
 'TAXI_OUT': 16.0,
 'WHEELS_OFF': 1913.0,
 'WHEELS_ON': 2019.0,
 'TAXI_IN': 10.0,
 'ARR_TIME': 2029.0,
 'ARR_DELAY': -8.0,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 0,
 'ACTUAL_ELAPSED_TIME': 92.0,
 'FLIGHTS': 1,
 'DISTANCE': 266,
 'CARRIER_DELAY': nan,
 'WEATHER_DELAY': nan,
 'NAS_DELAY': nan,
 'SECURITY_DELAY': nan,
 'LATE_AIRCRAFT_DELAY': nan,
 'Unnamed: 31': None,
 'CARRIER_NAME': 'JetBlue Airways',
 'AIRLINE_NAME': 'JetBlue Airways: B6',
 'ORIGIN_AIRPORT_NAME': 'Burlington, VT: Burlington International',
 'CANCELLATION_REASON': 'No input'}

# Homework 2

In [15]:
#What are the unique airlines in the list of flights?
unique_airline = { row['AIRLINE_NAME'] for row in flights}

In [16]:
unique_airline

{'Alaska Airlines Inc.: AS',
 'American Airlines Inc.: AA',
 'Delta Air Lines Inc.: DL',
 'ExpressJet Airlines Inc.: EV',
 'Frontier Airlines Inc.: F9',
 'Hawaiian Airlines Inc.: HA',
 'JetBlue Airways: B6',
 'SkyWest Airlines Inc.: OO',
 'Southwest Airlines Co.: WN',
 'Spirit Air Lines: NK',
 'United Air Lines Inc.: UA',
 'Virgin America: VX'}

In [17]:
x = { row['DEP_DELAY'] for row in flights}

In [18]:
long_delay = [ row for row in flights if row['DEP_DELAY'] > 1000]

In [19]:
len(long_delay)

55

In [20]:
long_delay_pick = [ row for row in flights if row['DEP_DELAY'] > 1400]

In [21]:
len(long_delay_pick)

6

In [22]:
long_delay_pick

[{'DAY_OF_MONTH': 7,
  'DAY_OF_WEEK': 2,
  'FL_DATE': '2017-02-07',
  'UNIQUE_CARRIER': 'AA',
  'AIRLINE_ID': 19805,
  'FL_NUM': 2195,
  'ORIGIN_AIRPORT_ID': 12441,
  'ORIGIN_CITY_NAME': 'Jackson, WY',
  'ORIGIN_STATE_NM': 'Wyoming',
  'DEST_AIRPORT_ID': 11298,
  'DEST_CITY_NAME': 'Dallas/Fort Worth, TX',
  'DEST_STATE_NM': 'Texas',
  'DEP_TIME': 1242.0,
  'DEP_DELAY': 1410.0,
  'TAXI_OUT': 16.0,
  'WHEELS_OFF': 1258.0,
  'WHEELS_ON': 1552.0,
  'TAXI_IN': 6.0,
  'ARR_TIME': 1558.0,
  'ARR_DELAY': 1382.0,
  'CANCELLED': 0,
  'CANCELLATION_CODE': None,
  'DIVERTED': 0,
  'ACTUAL_ELAPSED_TIME': 136.0,
  'FLIGHTS': 1,
  'DISTANCE': 1047,
  'CARRIER_DELAY': 76.0,
  'WEATHER_DELAY': 0.0,
  'NAS_DELAY': 0.0,
  'SECURITY_DELAY': 0.0,
  'LATE_AIRCRAFT_DELAY': 1306.0,
  'Unnamed: 31': None,
  'CARRIER_NAME': 'American Airlines Inc.',
  'AIRLINE_NAME': 'American Airlines Inc.: AA',
  'ORIGIN_AIRPORT_NAME': 'Jackson, WY: Jackson Hole',
  'CANCELLATION_REASON': 'No input'},
 {'DAY_OF_MONTH': 10,
  

In [23]:
[5,1,2,6,5,8]

[5, 1, 2, 6, 5, 8]

In [24]:
for i in [5,1,2,6,5,8]:
    print(i)

5
1
2
6
5
8


In [25]:
#Which flight had the longest departure delay?
#i iteration should relate to all lines 
longest_delay=0
max_idx=None

for i in long_delay_pick:
    #version 1: two lookups
    if longest_delay < i['DEP_DELAY']:
        longest_delay = i['DEP_DELAY']
        max_idx = i
        
    #version 2
    
    
    #version 3
        
    #print('Maximum value:' ,longest_delay, 'at index:', max_idx)

In [26]:
max_idx

{'DAY_OF_MONTH': 25,
 'DAY_OF_WEEK': 6,
 'FL_DATE': '2017-02-25',
 'UNIQUE_CARRIER': 'EV',
 'AIRLINE_ID': 20366,
 'FL_NUM': 5160,
 'ORIGIN_AIRPORT_ID': 10135,
 'ORIGIN_CITY_NAME': 'Allentown/Bethlehem/Easton, PA',
 'ORIGIN_STATE_NM': 'Pennsylvania',
 'DEST_AIRPORT_ID': 11433,
 'DEST_CITY_NAME': 'Detroit, MI',
 'DEST_STATE_NM': 'Michigan',
 'DEP_TIME': 2033.0,
 'DEP_DELAY': 1668.0,
 'TAXI_OUT': 19.0,
 'WHEELS_OFF': 2052.0,
 'WHEELS_ON': nan,
 'TAXI_IN': nan,
 'ARR_TIME': nan,
 'ARR_DELAY': nan,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 1,
 'ACTUAL_ELAPSED_TIME': nan,
 'FLIGHTS': 1,
 'DISTANCE': 425,
 'CARRIER_DELAY': nan,
 'WEATHER_DELAY': nan,
 'NAS_DELAY': nan,
 'SECURITY_DELAY': nan,
 'LATE_AIRCRAFT_DELAY': nan,
 'Unnamed: 31': None,
 'CARRIER_NAME': 'ExpressJet Airlines Inc.',
 'AIRLINE_NAME': 'ExpressJet Airlines Inc.: EV',
 'ORIGIN_AIRPORT_NAME': 'Allentown/Bethlehem/Easton, PA: Lehigh Valley International',
 'CANCELLATION_REASON': 'No input'}

In [27]:
#find the min
temps = [
    {'temp': 80, 'name': 'slc'},
    {'temp': 75, 'name': 'omaha'}, 
    {'temp': 90, 'name': 'miami'},
]

In [28]:
max(flights, key=lambda d: d['DEP_DELAY'])

{'DAY_OF_MONTH': 25,
 'DAY_OF_WEEK': 6,
 'FL_DATE': '2017-02-25',
 'UNIQUE_CARRIER': 'EV',
 'AIRLINE_ID': 20366,
 'FL_NUM': 5160,
 'ORIGIN_AIRPORT_ID': 10135,
 'ORIGIN_CITY_NAME': 'Allentown/Bethlehem/Easton, PA',
 'ORIGIN_STATE_NM': 'Pennsylvania',
 'DEST_AIRPORT_ID': 11433,
 'DEST_CITY_NAME': 'Detroit, MI',
 'DEST_STATE_NM': 'Michigan',
 'DEP_TIME': 2033.0,
 'DEP_DELAY': 1668.0,
 'TAXI_OUT': 19.0,
 'WHEELS_OFF': 2052.0,
 'WHEELS_ON': nan,
 'TAXI_IN': nan,
 'ARR_TIME': nan,
 'ARR_DELAY': nan,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 1,
 'ACTUAL_ELAPSED_TIME': nan,
 'FLIGHTS': 1,
 'DISTANCE': 425,
 'CARRIER_DELAY': nan,
 'WEATHER_DELAY': nan,
 'NAS_DELAY': nan,
 'SECURITY_DELAY': nan,
 'LATE_AIRCRAFT_DELAY': nan,
 'Unnamed: 31': None,
 'CARRIER_NAME': 'ExpressJet Airlines Inc.',
 'AIRLINE_NAME': 'ExpressJet Airlines Inc.: EV',
 'ORIGIN_AIRPORT_NAME': 'Allentown/Bethlehem/Easton, PA: Lehigh Valley International',
 'CANCELLATION_REASON': 'No input'}

# Homework 3

# H3 Q1

In [29]:
from collections import defaultdict, Counter
# difference between a dict and defaultdict is that defaultdict overrides
# .__missing__() and it adds .default_factory. Writable instance var that 
# needs to be provided at the time of instantiation
# https://realpython.com/python-defaultdict/#using-the-python-defaultdict-type

In [30]:
{ row['CANCELLATION_REASON'] for row in flights }

{'Carrier', 'National Air System', 'No input', 'Weather'}

In [31]:
#Which carrier had the most time delay b/c of weather?
weather = [ row for row in flights if row['CANCELLATION_REASON'] == 'Weather']

In [32]:
len(weather)

4701

In [33]:
weather_delay_1 = [ row for row in flights if row['WEATHER_DELAY'] > 0]

In [34]:
len(weather_delay_1)

3330

In [35]:
weather_delay_1[0]

{'DAY_OF_MONTH': 1,
 'DAY_OF_WEEK': 3,
 'FL_DATE': '2017-02-01',
 'UNIQUE_CARRIER': 'F9',
 'AIRLINE_ID': 20436,
 'FL_NUM': 552,
 'ORIGIN_AIRPORT_ID': 14679,
 'ORIGIN_CITY_NAME': 'San Diego, CA',
 'ORIGIN_STATE_NM': 'California',
 'DEST_AIRPORT_ID': 11292,
 'DEST_CITY_NAME': 'Denver, CO',
 'DEST_STATE_NM': 'Colorado',
 'DEP_TIME': 1336.0,
 'DEP_DELAY': 436.0,
 'TAXI_OUT': 16.0,
 'WHEELS_OFF': 1352.0,
 'WHEELS_ON': 1649.0,
 'TAXI_IN': 26.0,
 'ARR_TIME': 1715.0,
 'ARR_DELAY': 441.0,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 0,
 'ACTUAL_ELAPSED_TIME': 159.0,
 'FLIGHTS': 1,
 'DISTANCE': 853,
 'CARRIER_DELAY': 0.0,
 'WEATHER_DELAY': 436.0,
 'NAS_DELAY': 5.0,
 'SECURITY_DELAY': 0.0,
 'LATE_AIRCRAFT_DELAY': 0.0,
 'Unnamed: 31': None,
 'CARRIER_NAME': 'Frontier Airlines Inc.',
 'AIRLINE_NAME': 'Frontier Airlines Inc.: F9',
 'ORIGIN_AIRPORT_NAME': 'San Diego, CA: San Diego International',
 'CANCELLATION_REASON': 'No input'}

In [36]:
weather_delay=0
max_idx=None

weather_delay_def_dict = defaultdict(list)
for i in weather_delay_1:
    if weather_delay < i['WEATHER_DELAY']:
        weather_delay = i['WEATHER_DELAY']
        max_idx = i

In [37]:
max_idx

{'DAY_OF_MONTH': 11,
 'DAY_OF_WEEK': 6,
 'FL_DATE': '2017-02-11',
 'UNIQUE_CARRIER': 'AA',
 'AIRLINE_ID': 19805,
 'FL_NUM': 2400,
 'ORIGIN_AIRPORT_ID': 11298,
 'ORIGIN_CITY_NAME': 'Dallas/Fort Worth, TX',
 'ORIGIN_STATE_NM': 'Texas',
 'DEST_AIRPORT_ID': 11503,
 'DEST_CITY_NAME': 'Eagle, CO',
 'DEST_STATE_NM': 'Colorado',
 'DEP_TIME': 1215.0,
 'DEP_DELAY': 1175.0,
 'TAXI_OUT': 26.0,
 'WHEELS_OFF': 1241.0,
 'WHEELS_ON': 1358.0,
 'TAXI_IN': 5.0,
 'ARR_TIME': 1403.0,
 'ARR_DELAY': 1203.0,
 'CANCELLED': 0,
 'CANCELLATION_CODE': None,
 'DIVERTED': 0,
 'ACTUAL_ELAPSED_TIME': 168.0,
 'FLIGHTS': 1,
 'DISTANCE': 721,
 'CARRIER_DELAY': 0.0,
 'WEATHER_DELAY': 1175.0,
 'NAS_DELAY': 28.0,
 'SECURITY_DELAY': 0.0,
 'LATE_AIRCRAFT_DELAY': 0.0,
 'Unnamed: 31': None,
 'CARRIER_NAME': 'American Airlines Inc.',
 'AIRLINE_NAME': 'American Airlines Inc.: AA',
 'ORIGIN_AIRPORT_NAME': 'Dallas/Fort Worth, TX: Dallas/Fort Worth International',
 'CANCELLATION_REASON': 'No input'}

In [38]:
max_idx['CARRIER_NAME']

'American Airlines Inc.'

# H3 Q2

In [39]:
# what are the top 10 weather delays in descending order?

In [40]:
len(weather_delay_1)

3330

In [41]:
weather_delay_1.sort(key=lambda i: i['WEATHER_DELAY'], reverse=True)

In [42]:
order = weather_delay_1[0:10]
order

[{'DAY_OF_MONTH': 11,
  'DAY_OF_WEEK': 6,
  'FL_DATE': '2017-02-11',
  'UNIQUE_CARRIER': 'AA',
  'AIRLINE_ID': 19805,
  'FL_NUM': 2400,
  'ORIGIN_AIRPORT_ID': 11298,
  'ORIGIN_CITY_NAME': 'Dallas/Fort Worth, TX',
  'ORIGIN_STATE_NM': 'Texas',
  'DEST_AIRPORT_ID': 11503,
  'DEST_CITY_NAME': 'Eagle, CO',
  'DEST_STATE_NM': 'Colorado',
  'DEP_TIME': 1215.0,
  'DEP_DELAY': 1175.0,
  'TAXI_OUT': 26.0,
  'WHEELS_OFF': 1241.0,
  'WHEELS_ON': 1358.0,
  'TAXI_IN': 5.0,
  'ARR_TIME': 1403.0,
  'ARR_DELAY': 1203.0,
  'CANCELLED': 0,
  'CANCELLATION_CODE': None,
  'DIVERTED': 0,
  'ACTUAL_ELAPSED_TIME': 168.0,
  'FLIGHTS': 1,
  'DISTANCE': 721,
  'CARRIER_DELAY': 0.0,
  'WEATHER_DELAY': 1175.0,
  'NAS_DELAY': 28.0,
  'SECURITY_DELAY': 0.0,
  'LATE_AIRCRAFT_DELAY': 0.0,
  'Unnamed: 31': None,
  'CARRIER_NAME': 'American Airlines Inc.',
  'AIRLINE_NAME': 'American Airlines Inc.: AA',
  'ORIGIN_AIRPORT_NAME': 'Dallas/Fort Worth, TX: Dallas/Fort Worth International',
  'CANCELLATION_REASON': 'No input

In [43]:
[ row['CARRIER_NAME'] for row in order]

['American Airlines Inc.',
 'United Air Lines Inc.',
 'SkyWest Airlines Inc.',
 'SkyWest Airlines Inc.',
 'United Air Lines Inc.',
 'SkyWest Airlines Inc.',
 'American Airlines Inc.',
 'SkyWest Airlines Inc.',
 'ExpressJet Airlines Inc.',
 'Hawaiian Airlines Inc.']

# H3Q2 attempt #2 lol

In [44]:
# what are the top 10 weather delays in descending order?
desc_list = Counter(row['CARRIER_NAME'] for row in weather_delay_1)

In [45]:
desc_list.most_common(10)

[('Delta Air Lines Inc.', 565),
 ('Southwest Airlines Co.', 560),
 ('SkyWest Airlines Inc.', 425),
 ('Hawaiian Airlines Inc.', 362),
 ('Virgin America', 348),
 ('United Air Lines Inc.', 344),
 ('American Airlines Inc.', 276),
 ('JetBlue Airways', 167),
 ('Alaska Airlines Inc.', 157),
 ('ExpressJet Airlines Inc.', 73)]